<a href="https://colab.research.google.com/github/rajathpatel23/Chord-implementation/blob/master/MobileNet_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import keras
from keras.applications.mobilenet import MobileNet
from keras.models import Model
from pickle import dump
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
from keras.regularizers import l2
from sklearn.externals import joblib

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
train_dict_1 = joblib.load('/content/drive/My Drive/Dataset/train_stft_data_1.pkl')
train_dict_2 = joblib.load('/content/drive/My Drive/Dataset/train_stft_data_2.pkl')

In [0]:
def get_tuning(X_in):
      tuned_out = []
      for x in range(len(X_in)):
            if X_in[x][0] >= 0.5:
                  tuned_out.append(1)
            else:
                  tuned_out.append(0)
      return tuned_out

def get_confusion_matrix(y_t, y_pred):
      tn, fp, fn, tp = confusion_matrix(y_t, y_pred).ravel()
      sensitivity = tp/(tp + fn)
      specificity = tn/(tn + fp)
      PPV = tp/(tp + fp)
      NPV = tn/(tn + fn)
      return sensitivity, specificity, PPV, NPV

def get_aucpr(y_t, y_pred):
      precision, recall, thres  = precision_recall_curve(y_t, y_pred)
      aucpr = auc(recall, precision)
      return aucpr


def get_processed_X(X):
      X_copy  = X
      for arr in range(len(X_copy)):
            a = np.min(X_copy[arr])
            b = np.max(X_copy[arr])
            X_copy[arr] = np.divide(np.subtract(X_copy[arr], a), (b-a))
      return X_copy 
#%%
def get_reshape(X):
      X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
      return X

In [0]:
X = np.concatenate((train_dict_1["stft"], train_dict_2["stft"]))
y = np.concatenate((train_dict_1["MT"], train_dict_2["MT"]))

In [0]:
del train_dict_1
del train_dict_2
X = get_processed_X(X)

In [20]:
X = np.stack((X,)*3, axis=-1)
print(X.shape)

(7683, 375, 148, 3)


In [21]:
model_1 = MobileNet(include_top=False, input_shape=(375, 148, 3))
# print(model_1.summary())

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [22]:
average_layer =GlobalAveragePooling2D()(model_1.output)
# flat1 =Flatten()(average_layer)
# class1 = Dense(1024, activation='relu')(average_layer)
class2 = Dense(512, activation='relu')(average_layer)
output = Dense(1, activation='sigmoid', activity_regularizer=l2(0.01))(class2)
model = Model(inputs = model_1.inputs, outputs = output)
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam')

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 375, 148, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 376, 149, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 187, 74, 32)       864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 187, 74, 32)       128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 187, 74, 32)       0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 187, 74, 32)       288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 187, 74, 32)       128 

In [23]:
from sklearn.utils import class_weight
from kekeras.callbacks import ModelCheckpoint, EarlyStopping
sample_weights = class_weight.compute_sample_weight(class_weight="balanced", y=y)
callbacks = [EarlyStopping(monitor='loss', min_delta=0, patience=2, verbose=0, mode='auto', baseline=None, restore_best_weights=True), 
             ModelCheckpoint(filepath='/concrete/drive/My drive/Dataset/mobilenet_best_model.h5', monitor='loss', save_best_only=True)]
history = model.fit(X, y,epochs=20,batch_size=64,shuffle=True, verbose=1, 
                    sample_weight=sample_weights, callbacks=callbacks)

Epoch 1/20
7683/7683 [==============================] - 105s 14ms/step - loss: 0.7864
Epoch 2/20
7683/7683 [==============================] - 97s 13ms/step - loss: 0.5801
Epoch 3/20
7683/7683 [==============================] - 97s 13ms/step - loss: 0.5490
Epoch 4/20
7683/7683 [==============================] - 97s 13ms/step - loss: 0.4607
Epoch 5/20
7683/7683 [==============================] - 97s 13ms/step - loss: 0.4362
Epoch 6/20
7683/7683 [==============================] - 98s 13ms/step - loss: 0.3820
Epoch 7/20
7683/7683 [==============================] - 97s 13ms/step - loss: 0.3328
Epoch 8/20
7683/7683 [==============================] - 96s 13ms/step - loss: 0.2883
Epoch 9/20
7683/7683 [==============================] - 97s 13ms/step - loss: 0.3832
Epoch 10/20
7683/7683 [==============================] - 98s 13ms/step - loss: 0.2892
Epoch 11/20
7683/7683 [==============================] - 98s 13ms/step - loss: 0.2412
Epoch 12/20
7683/7683 [==============================] - 96s 1

In [0]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, cohen_kappa_score, auc, roc_curve
from sklearn.metrics import average_precision_score, confusion_matrix, precision_recall_curve, classification_report
def performance_analysis(y_2, y_test_1):      
      tuned_y_2 = get_tuning(y_2)
      roc_auc_test = roc_auc_score(y_test_1, y_2)
      print("The test roc is : ", roc_auc_test)
      aucpr_test = get_aucpr(y_test_1, y_2)
      print("The test aucpr is: ", aucpr_test)
      sense_test_, specs_test_, PPV_test_, NPV_test_ = get_confusion_matrix(y_test_1, tuned_y_2)
      print("The test sensitivity is: ", sense_test_)
      print("The test specificity is: ", specs_test_)
      print("The test PPV is: ", PPV_test_)
      print("The test NPV is: ", NPV_test_)
      return True

In [0]:
test_1 = joblib.load('/content/drive/My Drive/Dataset/test_stft_data_1.pkl')
test_2 = joblib.load('/content/drive/My Drive/Dataset/test_stft_data_2.pkl')

In [26]:
X = np.concatenate((test_1["stft"], test_2["stft"]))
y_test = np.concatenate((test_1["MT"], test_2["MT"]))
del test_1
del test_2
X = get_processed_X(X)
X = np.stack((X,)*3, axis=-1)
print(X.shape)

(6831, 375, 148, 3)


In [27]:
y_pred_MT = model.predict(X)
performance_analysis(y_pred_MT, y_test)

The test roc is :  0.8980895029849487
The test aucpr is:  0.18912871094105335
The test sensitivity is:  0.5434782608695652
The test specificity is:  0.9475571492604213
The test PPV is:  0.176056338028169
The test NPV is:  0.9901639344262295


True

In [0]:
model.save('/content/drive/My Drive/Dataset/MobileNet_20_64_MT_STFT.h5')